### 参考文献
萩原淳一郎, 瓜生真也, 牧山幸史. 基礎からわかる時系列分析―Rで実践するカルマンフィルタ・MCMC・粒子フィルタ―. Data Science Library

- - - 

# 状態空間モデル(Space State Model: SSM)

* 推定対象は値ではなく確率分布
* 直接観測されるデータに加えて、直感的には観測されない潜在的な確率変数(=状態)を導入。
* 状態はマルコフ性(前時点のみと関連がある)を仮定する。
* ある時点の観測値はその時点の状態によってのみ決まる。

## 0. 定義

方程式によって表現すると

$$
\left\{
\begin{array}{ll}
\mathbf{x}_{t} = f(\mathbf{x}_{t_1}, \mathbf{w}_{t}) & \dots \text{状態方程式orシステム方程式}\\
y_{t} = g(\mathbf{x}_{t}+v_{t}) & \dots 観測方程式
\end{array}
\right.
$$
ただし$f(\cdot), g(\cdot)$は任意の関数、$\mathbf{w}_{t}, v_{t}$はそれぞれ状態雑音、観測雑音と呼ばれるホワイトノイズ。

## 1. 分類

$f(\cdot), g(\cdot)$の種類によってモデルの形状に名前がついている

|  | $\mathbf{w}_{t}, v_{t}$のいずれかが非ガウス分布 | $\mathbf{w}_{t}, v_{t}$がともにガウス分布 |
| - | - | - |
| $f(\cdot), g(\cdot)$のいずれかが非線形 | 非線形・非ガウス型状態空間モデル | 非線形・ガウス型状態空間モデル |
| $f(\cdot), g(\cdot)$がともに線形 | 線形・非ガウス型状態空間モデル | 線形・ガウス型状態空間モデル <br>or 動的線形モデル(Dynamic Linear Model: DLM) |  

特に
### 線形・ガウス型状態空間モデル
$$
\left\{
\begin{array}{ll}
\mathbf{x}_{t} = \mathbf{G}_{t}\mathbf{x}_{t-1}+\mathbf{w}_{t},  & \mathbf{w}_{t} \sim N(\mathbf{0},\mathbf{W}_{t}), \quad
\mathbf{x}_{0} \sim N(\mathbf{m}_{0}, \mathbf{C}_{0})\\
y_{t} = \mathbf{F}_{t}\mathbf{x}_{t}+v_{t},  & v_{t} \sim N(0, V_{t})
\end{array}
\right.
$$
もしくは
$$
\left\{
\begin{array}{ll}
p(\mathbf{x}_{t}|\mathbf{x}_{t-1}) = N(\mathbf{G}_{t}\mathbf{x}_{t-1}, \mathbf{W}_{t})\\
p(\mathbf{x}_{0}) = N(\mathbf{m}_{0}, \mathbf{C}_{0})\\
p(y_{t}|\mathbf{x}_{t}) = N(\mathbf{F}_{t}\mathbf{x}_{t-1}, V_{t})
\end{array}
\right.
$$
ここで、  
$\mathbf{G}_{t}$は$p \times p$の**状態遷移行列**、$\mathbf{F}_{t}$は$1 \times p$の**観測行列**、$\mathbf{W}_{t}$は$p \times p$の状態雑音の共分散行列、$\mathbf{V}_{t}$は観測雑音の分散、
$\mathbf{m}_{0}$は事前分布における$p$次元の平均ベクトル、$\mathbf{C}_{0}$は事前分布における$p \times p$の共分散行列とする。

## 2. 状態の推定

## 3. パラメータの推定

SSMのパラメータをまとめて$\mathbf{\theta}$とする

### 3-1. パラメータを確率分布として考えない場合(頻度論的)

#### 最尤推定  
尤度は
$$
\begin{align*}
p(y_{1}, y_{2}, \dots, y_{T};\mathbf{\theta}) &= p(y_{1:T};\mathbf{\theta}) \\
&= p(y_{T}, y_{1:T-1}; \mathbf{\theta}) \\
&= p(y_{T}|y_{1:T-1};\mathbf{\theta})p(y_{1:T-1};\mathbf{\theta}) \\
&= \prod_{t=1}^{T} p(y_{t}|y_{1:t-1};T) \\
\end{align*}
$$
ここで$y_{1:0}$は存在しないので空集合とすると
$p(y_{1}|y_{1:0};\mathbf{\theta}) = p(p_{1}|\emptyset;\mathbf{\theta}) = p(y_{1};\mathbf{\theta})$。  
対数とると
$$
\begin{align*}
l(\mathbf{\theta}) &= \log{p(y_{1}, y_{2}, \dots, y_{T};\mathbf{\theta})} \\
&= \sum_{t=1}^{T} \log{p(y_{t}|y_{1:t-1};T)}
\end{align*}
$$

従って最尤推定値$\mathbf{\hat{\theta}}$は
$$
\begin{align*}
\mathbf{\hat{\theta}} &= \underset{\mathbf{\hat{\theta}}}{\text{argmax }} l(\mathbf{\theta}) \\
&= \underset{\mathbf{\hat{\theta}}}{\text{argmax }} \log{p(y_{1:T}|\mathbf{\theta})}\\
&= \underset{\mathbf{\hat{\theta}}}{\text{argmax }} \sum_{t=1}^{T} \log{p(y_{t}|y_{1:t-1};T)}
\end{align*}
$$

### 3-2. パラメータを確率分布として考える場合(ベイズ的)

#### MAP推定(周辺事後分布を最大化する)

$$
\begin{align*}
\underset{\mathbf{\theta}}{\text{argmax }}p(\mathbf{\theta}|y_{1:T})
&=  \underset{\mathbf{\theta}}{\text{argmax }}\frac{p(y_{1:T}|\mathbf{\theta})p(\mathbf{\theta})}{p(y_{1:T})}  \\
&\propto \underset{\mathbf{\theta}}{\text{argmax }}p(y_{1:T}|\mathbf{\theta})p(\mathbf{\theta})
\end{align*}
$$
対数取ると
$$
\underset{\mathbf{\theta}}{\text{argmax }} \log{p(\mathbf{\theta}|y_{1:T})} \propto \underset{\mathbf{\theta}}{\text{argmax }} \{ \log{p(y_{1:T}|\mathbf{\theta})} + \log{p(\mathbf{\theta})}\}
$$

## 4. 解法

ウィナーフィルタ、カルマンフィルタ、 MCMC、粒子フィルタ